In [65]:
SRC = [
    "TheDevOpsHandbookSecondEdition",
    "DavidNBlankEdelmanSeekingSREConversation",
]

DST = "DevopsTest1"

In [ ]:
! pip install pinecone-client

In [56]:
import os

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = "asia-southeast1-gcp"
PINECONE_INDEX = "aisystant"
PINECONE_DIMS = 1536

STEP_SIZE = 10

In [15]:
import pinecone

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

In [18]:
index = pinecone.Index(PINECONE_INDEX)

In [70]:
def all_docs(src):
    result = index.query(
        vector=[0.0 for _ in range(PINECONE_DIMS)],
        top_k=10000,
        namespace=src,
    )
    return list(map(lambda x: x["id"], result["matches"]))

def convert_vectors(docs):
    return list(map(lambda x: (x["id"], x["values"], x["metadata"]), docs["vectors"].values()))

def move_docs(src, dst):
    all_ids = all_docs(src)
    # split all_ids into chunks of size STEP_SIZE
    for i in range(0, len(all_ids), STEP_SIZE):
        print(f"Processing {src}: {i} to {i + STEP_SIZE} of {len(all_ids)}")
        ids = all_ids[i : i + STEP_SIZE]
        docs = index.fetch(ids, namespace=src)
        vectors = convert_vectors(docs)
        #print(vectors)
        index.upsert(vectors, namespace=dst)

In [ ]:
for src in SRC:
    print("Starting to move docs from", src, "to", DST)
    move_docs(src, DST)
    print("Done moving docs from", src, "to", DST)